In [1]:
import os
# Find the latest version of spark 3.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.1'
spark_version = 'spark-3.2.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:8 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:9 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:10 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [73.9 kB]
Get:12 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:13 https://developer.download.nvi

In [2]:
# For connection to Postgres
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2022-01-01 20:41:37--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  1.30MB/s    in 0.7s    

2022-01-01 20:41:39 (1.30 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [3]:
# Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("zillow_regression").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [4]:
# Read in data from Zillow
from pyspark import SparkFiles
url ="https://raw.githubusercontent.com/designergal3002/Project-4-Team/main/test_work/resources/Metro_zhvi_uc_sfrcondo_tier_0.33_0.67_sm_sa_month.csv"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("Metro_zhvi_uc_sfrcondo_tier_0.33_0.67_sm_sa_month.csv"), sep=',', header=True)

# Show DataFrame
df.show()

+--------+--------+--------------------+----------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+----------+----------+----------+---------+---------+---------+---------+---------+---------+---------+---------+---------+----------+----------+----------+---------+---------+---------+---------+---------+---------+---------+---------+---------+----------+----------+----------+---------+---------+---------+---------+---------+---------+---------+---------+---------+----------+----------+----------+---------+---------+---------+---------+---------+---------+---------+---------+---------+----------+----------+----------+---------+---------+---------+---------+---------+---------+---------+---------+---------+----------+----------+----------+---------+---------+---------+---------+---------+---------+---------+---------+---------+----------+----------+----------+---------+---------+---------+---------+---------+---------+---------+--------

In [5]:
# Read in fed rate data
url ="https://raw.githubusercontent.com/designergal3002/Project-4-Team/main/test_work/resources/FEDFUNDS.csv"
spark.sparkContext.addFile(url)
fed_df = spark.read.csv(SparkFiles.get("FEDFUNDS.csv"), sep=',', header=True)

# Show DataFrame
fed_df.show()

+----------+--------+
|      DATE|FEDFUNDS|
+----------+--------+
|1954-07-01|    0.80|
|1954-08-01|    1.22|
|1954-09-01|    1.07|
|1954-10-01|    0.85|
|1954-11-01|    0.83|
|1954-12-01|    1.28|
|1955-01-01|    1.39|
|1955-02-01|    1.29|
|1955-03-01|    1.35|
|1955-04-01|    1.43|
|1955-05-01|    1.43|
|1955-06-01|    1.64|
|1955-07-01|    1.68|
|1955-08-01|    1.96|
|1955-09-01|    2.18|
|1955-10-01|    2.24|
|1955-11-01|    2.35|
|1955-12-01|    2.48|
|1956-01-01|    2.45|
|1956-02-01|    2.50|
+----------+--------+
only showing top 20 rows



In [6]:
# Limit Zillow data to past 10 years
c_names = df.schema.names
c_names = c_names[5:148]
for name in c_names:
  df = df.drop(name)
df.show()

+--------+--------+--------------------+----------+---------+----------+---------+---------+---------+---------+---------+---------+---------+---------+---------+----------+----------+----------+---------+---------+---------+---------+---------+---------+---------+---------+---------+----------+----------+----------+---------+---------+---------+---------+---------+---------+---------+---------+---------+----------+----------+----------+---------+---------+---------+---------+---------+---------+---------+---------+---------+----------+----------+----------+---------+---------+---------+---------+---------+---------+---------+---------+---------+----------+----------+----------+---------+---------+---------+---------+---------+---------+---------+---------+---------+----------+----------+----------+---------+---------+---------+---------+---------+---------+---------+---------+---------+----------+----------+----------+---------+---------+---------+---------+---------+---------+-------

In [7]:
# Limit fed rate data to past 10 years and split into two lists
dates = fed_df.select("DATE").rdd.flatMap(lambda x: x).collect()
dates = dates[689:]
funds = fed_df.select("FEDFUNDS").rdd.flatMap(lambda x: x).collect()
funds = funds[689:]

In [9]:
# Convert Zillow data to Pandas and add fed rate data
import pandas as pd
full_df = df.toPandas()
for i in range(0, len(dates)):
  full_df[f"FedFund_{dates[i]}"] = funds[i]
full_df.head(5)

,RegionID,SizeRank,RegionName,RegionType,StateName,12/31/2011,1/31/2012,2/29/2012,3/31/2012,4/30/2012,5/31/2012,6/30/2012,7/31/2012,8/31/2012,9/30/2012,10/31/2012,11/30/2012,12/31/2012,1/31/2013,2/28/2013,3/31/2013,4/30/2013,5/31/2013,6/30/2013,7/31/2013,8/31/2013,9/30/2013,10/31/2013,11/30/2013,12/31/2013,1/31/2014,2/28/2014,3/31/2014,4/30/2014,5/31/2014,6/30/2014,7/31/2014,8/31/2014,9/30/2014,10/31/2014,...,FedFund_2018-08-01,FedFund_2018-09-01,FedFund_2018-10-01,FedFund_2018-11-01,FedFund_2018-12-01,FedFund_2019-01-01,FedFund_2019-02-01,FedFund_2019-03-01,FedFund_2019-04-01,FedFund_2019-05-01,FedFund_2019-06-01,FedFund_2019-07-01,FedFund_2019-08-01,FedFund_2019-09-01,FedFund_2019-10-01,FedFund_2019-11-01,FedFund_2019-12-01,FedFund_2020-01-01,FedFund_2020-02-01,FedFund_2020-03-01,FedFund_2020-04-01,FedFund_2020-05-01,FedFund_2020-06-01,FedFund_2020-07-01,FedFund_2020-08-01,FedFund_2020-09-01,FedFund_2020-10-01,FedFund_2020-11-01,FedFund_2020-12-01,FedFund_2021-01-01,FedFund_2021-02-01,FedFund_2021-03-01,FedFund_2021-04-01,FedFund_2021-05-01,FedFund_2021-06-01,FedFund_2021-07-01,FedFund_2021-08-01,FedFund_2021-09-01,FedFund_2021-10-01,FedFund_2021-11-01
0,102001,0,United States,Country,None,163382,163154,163045,163213,163536,164012,164463,164925,165364,165851,166430,167138,167933,168903,169987,171226,172478,173810,175166,176526,177786,178987,180139,181216,182246,183268,184320,185190,185904,186554,187221,187808,188212,188591,189078,...,1.91,1.95,2.19,2.20,2.27,2.40,2.40,2.41,2.42,2.39,2.38,2.40,2.13,2.04,1.83,1.55,1.55,1.55,1.58,0.65,0.05,0.05,0.08,0.09,0.10,0.09,0.09,0.09,0.09,0.09,0.08,0.07,0.07,0.06,0.08,0.10,0.09,0.08,0.08,0.08
1,394913,1,"New York, NY",Msa,NY,379106,377685,376397,375334,374632,374365,374334,374125,373795,373417,373126,373283,373709,374372,375262,376370,378093,380074,381978,383690,385154,386705,388410,390201,391767,393628,395304,397261,398975,400673,402077,402925,403531,404044,404522,...,1.91,1.95,2.19,2.20,2.27,2.40,2.40,2.41,2.42,2.39,2.38,2.40,2.13,2.04,1.83,1.55,1.55,1.55,1.58,0.65,0.05,0.05,0.08,0.09,0.10,0.09,0.09,0.09,0.09,0.09,0.08,0.07,0.07,0.06,0.08,0.10,0.09,0.08,0.08,0.08
2,753899,2,"Los Angeles-Long Beach-Anaheim, CA",Msa,CA,396483,395863,395124,394448,394961,396043,397718,400313,402353,404749,408418,412781,419227,425560,434772,443798,453651,461761,470223,477598,485358,491880,497062,502796,507853,513535,516012,516986,516275,516300,516441,517287,515847,515388,515875,...,1.91,1.95,2.19,2.20,2.27,2.40,2.40,2.41,2.42,2.39,2.38,2.40,2.13,2.04,1.83,1.55,1.55,1.55,1.58,0.65,0.05,0.05,0.08,0.09,0.10,0.09,0.09,0.09,0.09,0.09,0.08,0.07,0.07,0.06,0.08,0.10,0.09,0.08,0.08,0.08
3,394463,3,"Chicago, IL",Msa,IL,181716,180629,178496,178349,178027,178483,178360,177549,176727,175763,176111,176499,176808,177536,178608,179590,180884,182116,184190,185855,187906,189501,191666,192638,193973,194711,196203,196976,197438,198107,199203,200075,200286,200614,200867,...,1.91,1.95,2.19,2.20,2.27,2.40,2.40,2.41,2.42,2.39,2.38,2.40,2.13,2.04,1.83,1.55,1.55,1.55,1.58,0.65,0.05,0.05,0.08,0.09,0.10,0.09,0.09,0.09,0.09,0.09,0.08,0.07,0.07,0.06,0.08,0.10,0.09,0.08,0.08,0.08
4,394514,4,"Dallas-Fort Worth, TX",Msa,TX,144594,144297,144682,145267,145949,146411,146988,147715,148149,148515,148837,149510,150185,150979,151664,152615,153583,154662,155772,156949,157887,158925,160084,161477,162718,163674,164734,165702,166669,167566,168658,169409,170150,170677,171673,...,1.91,1.95,2.19,2.20,2.27,2.40,2.40,2.41,2.42,2.39,2.38,2.40,2.13,2.04,1.83,1.55,1.55,1.55,1.58,0.65,0.05,0.05,0.08,0.09,0.10,0.09,0.09,0.09,0.09,0.09,0.08,0.07,0.07,0.06,0.08,0.10,0.09,0.08,0.08,0.08


In [10]:
# Show number of rows
full_df.shape[0]

908

In [11]:
# Remove duplicates and NAN values
full_df = full_df.drop_duplicates()
full_df = full_df.dropna()
full_df.head(5)

,RegionID,SizeRank,RegionName,RegionType,StateName,12/31/2011,1/31/2012,2/29/2012,3/31/2012,4/30/2012,5/31/2012,6/30/2012,7/31/2012,8/31/2012,9/30/2012,10/31/2012,11/30/2012,12/31/2012,1/31/2013,2/28/2013,3/31/2013,4/30/2013,5/31/2013,6/30/2013,7/31/2013,8/31/2013,9/30/2013,10/31/2013,11/30/2013,12/31/2013,1/31/2014,2/28/2014,3/31/2014,4/30/2014,5/31/2014,6/30/2014,7/31/2014,8/31/2014,9/30/2014,10/31/2014,...,FedFund_2018-08-01,FedFund_2018-09-01,FedFund_2018-10-01,FedFund_2018-11-01,FedFund_2018-12-01,FedFund_2019-01-01,FedFund_2019-02-01,FedFund_2019-03-01,FedFund_2019-04-01,FedFund_2019-05-01,FedFund_2019-06-01,FedFund_2019-07-01,FedFund_2019-08-01,FedFund_2019-09-01,FedFund_2019-10-01,FedFund_2019-11-01,FedFund_2019-12-01,FedFund_2020-01-01,FedFund_2020-02-01,FedFund_2020-03-01,FedFund_2020-04-01,FedFund_2020-05-01,FedFund_2020-06-01,FedFund_2020-07-01,FedFund_2020-08-01,FedFund_2020-09-01,FedFund_2020-10-01,FedFund_2020-11-01,FedFund_2020-12-01,FedFund_2021-01-01,FedFund_2021-02-01,FedFund_2021-03-01,FedFund_2021-04-01,FedFund_2021-05-01,FedFund_2021-06-01,FedFund_2021-07-01,FedFund_2021-08-01,FedFund_2021-09-01,FedFund_2021-10-01,FedFund_2021-11-01
1,394913,1,"New York, NY",Msa,NY,379106,377685,376397,375334,374632,374365,374334,374125,373795,373417,373126,373283,373709,374372,375262,376370,378093,380074,381978,383690,385154,386705,388410,390201,391767,393628,395304,397261,398975,400673,402077,402925,403531,404044,404522,...,1.91,1.95,2.19,2.20,2.27,2.40,2.40,2.41,2.42,2.39,2.38,2.40,2.13,2.04,1.83,1.55,1.55,1.55,1.58,0.65,0.05,0.05,0.08,0.09,0.10,0.09,0.09,0.09,0.09,0.09,0.08,0.07,0.07,0.06,0.08,0.10,0.09,0.08,0.08,0.08
2,753899,2,"Los Angeles-Long Beach-Anaheim, CA",Msa,CA,396483,395863,395124,394448,394961,396043,397718,400313,402353,404749,408418,412781,419227,425560,434772,443798,453651,461761,470223,477598,485358,491880,497062,502796,507853,513535,516012,516986,516275,516300,516441,517287,515847,515388,515875,...,1.91,1.95,2.19,2.20,2.27,2.40,2.40,2.41,2.42,2.39,2.38,2.40,2.13,2.04,1.83,1.55,1.55,1.55,1.58,0.65,0.05,0.05,0.08,0.09,0.10,0.09,0.09,0.09,0.09,0.09,0.08,0.07,0.07,0.06,0.08,0.10,0.09,0.08,0.08,0.08
3,394463,3,"Chicago, IL",Msa,IL,181716,180629,178496,178349,178027,178483,178360,177549,176727,175763,176111,176499,176808,177536,178608,179590,180884,182116,184190,185855,187906,189501,191666,192638,193973,194711,196203,196976,197438,198107,199203,200075,200286,200614,200867,...,1.91,1.95,2.19,2.20,2.27,2.40,2.40,2.41,2.42,2.39,2.38,2.40,2.13,2.04,1.83,1.55,1.55,1.55,1.58,0.65,0.05,0.05,0.08,0.09,0.10,0.09,0.09,0.09,0.09,0.09,0.08,0.07,0.07,0.06,0.08,0.10,0.09,0.08,0.08,0.08
4,394514,4,"Dallas-Fort Worth, TX",Msa,TX,144594,144297,144682,145267,145949,146411,146988,147715,148149,148515,148837,149510,150185,150979,151664,152615,153583,154662,155772,156949,157887,158925,160084,161477,162718,163674,164734,165702,166669,167566,168658,169409,170150,170677,171673,...,1.91,1.95,2.19,2.20,2.27,2.40,2.40,2.41,2.42,2.39,2.38,2.40,2.13,2.04,1.83,1.55,1.55,1.55,1.58,0.65,0.05,0.05,0.08,0.09,0.10,0.09,0.09,0.09,0.09,0.09,0.08,0.07,0.07,0.06,0.08,0.10,0.09,0.08,0.08,0.08
5,394974,5,"Philadelphia, PA",Msa,PA,209291,208533,207820,207529,207511,207453,206970,206278,205605,205198,204874,204793,204715,205011,205332,205884,206494,207395,208332,209225,209792,210405,210845,211414,211757,212345,212912,213284,213134,213149,213319,213603,213470,213295,213218,...,1.91,1.95,2.19,2.20,2.27,2.40,2.40,2.41,2.42,2.39,2.38,2.40,2.13,2.04,1.83,1.55,1.55,1.55,1.58,0.65,0.05,0.05,0.08,0.09,0.10,0.09,0.09,0.09,0.09,0.09,0.08,0.07,0.07,0.06,0.08,0.10,0.09,0.08,0.08,0.08


In [12]:
# Show number of rows
full_df.shape[0]

724

In [25]:
from io import IncrementalNewlineDecoder
# Set up Target as most recent price (y) and Features as rest of dataframe (X)
y = full_df['11/30/2021']
X = full_df.drop('11/30/2021', axis=1)

In [26]:
# Convert categorical data to numeric
X = pd.get_dummies(data=X, columns=['RegionName', 'RegionType', 'StateName'])
X.head(5)

,RegionID,SizeRank,12/31/2011,1/31/2012,2/29/2012,3/31/2012,4/30/2012,5/31/2012,6/30/2012,7/31/2012,8/31/2012,9/30/2012,10/31/2012,11/30/2012,12/31/2012,1/31/2013,2/28/2013,3/31/2013,4/30/2013,5/31/2013,6/30/2013,7/31/2013,8/31/2013,9/30/2013,10/31/2013,11/30/2013,12/31/2013,1/31/2014,2/28/2014,3/31/2014,4/30/2014,5/31/2014,6/30/2014,7/31/2014,8/31/2014,9/30/2014,10/31/2014,11/30/2014,12/31/2014,1/31/2015,...,StateName_HI,StateName_IA,StateName_ID,StateName_IL,StateName_IN,StateName_KS,StateName_KY,StateName_LA,StateName_MA,StateName_MD,StateName_ME,StateName_MI,StateName_MN,StateName_MO,StateName_MS,StateName_MT,StateName_NC,StateName_ND,StateName_NE,StateName_NH,StateName_NJ,StateName_NM,StateName_NV,StateName_NY,StateName_OH,StateName_OK,StateName_OR,StateName_PA,StateName_RI,StateName_SC,StateName_SD,StateName_TN,StateName_TX,StateName_UT,StateName_VA,StateName_VT,StateName_WA,StateName_WI,StateName_WV,StateName_WY
1,394913,1,379106,377685,376397,375334,374632,374365,374334,374125,373795,373417,373126,373283,373709,374372,375262,376370,378093,380074,381978,383690,385154,386705,388410,390201,391767,393628,395304,397261,398975,400673,402077,402925,403531,404044,404522,404942,405587,406288,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,753899,2,396483,395863,395124,394448,394961,396043,397718,400313,402353,404749,408418,412781,419227,425560,434772,443798,453651,461761,470223,477598,485358,491880,497062,502796,507853,513535,516012,516986,516275,516300,516441,517287,515847,515388,515875,519059,522838,527430,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,394463,3,181716,180629,178496,178349,178027,178483,178360,177549,176727,175763,176111,176499,176808,177536,178608,179590,180884,182116,184190,185855,187906,189501,191666,192638,193973,194711,196203,196976,197438,198107,199203,200075,200286,200614,200867,201323,201851,202343,...,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,394514,4,144594,144297,144682,145267,145949,146411,146988,147715,148149,148515,148837,149510,150185,150979,151664,152615,153583,154662,155772,156949,157887,158925,160084,161477,162718,163674,164734,165702,166669,167566,168658,169409,170150,170677,171673,172947,174399,176002,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
5,394974,5,209291,208533,207820,207529,207511,207453,206970,206278,205605,205198,204874,204793,204715,205011,205332,205884,206494,207395,208332,209225,209792,210405,210845,211414,211757,212345,212912,213284,213134,213149,213319,213603,213470,213295,213218,213394,213721,214042,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0


In [28]:
# Drop RegionID
X = X.drop('RegionID', axis=1)
X.head(5)

,SizeRank,12/31/2011,1/31/2012,2/29/2012,3/31/2012,4/30/2012,5/31/2012,6/30/2012,7/31/2012,8/31/2012,9/30/2012,10/31/2012,11/30/2012,12/31/2012,1/31/2013,2/28/2013,3/31/2013,4/30/2013,5/31/2013,6/30/2013,7/31/2013,8/31/2013,9/30/2013,10/31/2013,11/30/2013,12/31/2013,1/31/2014,2/28/2014,3/31/2014,4/30/2014,5/31/2014,6/30/2014,7/31/2014,8/31/2014,9/30/2014,10/31/2014,11/30/2014,12/31/2014,1/31/2015,2/28/2015,...,StateName_HI,StateName_IA,StateName_ID,StateName_IL,StateName_IN,StateName_KS,StateName_KY,StateName_LA,StateName_MA,StateName_MD,StateName_ME,StateName_MI,StateName_MN,StateName_MO,StateName_MS,StateName_MT,StateName_NC,StateName_ND,StateName_NE,StateName_NH,StateName_NJ,StateName_NM,StateName_NV,StateName_NY,StateName_OH,StateName_OK,StateName_OR,StateName_PA,StateName_RI,StateName_SC,StateName_SD,StateName_TN,StateName_TX,StateName_UT,StateName_VA,StateName_VT,StateName_WA,StateName_WI,StateName_WV,StateName_WY
1,1,379106,377685,376397,375334,374632,374365,374334,374125,373795,373417,373126,373283,373709,374372,375262,376370,378093,380074,381978,383690,385154,386705,388410,390201,391767,393628,395304,397261,398975,400673,402077,402925,403531,404044,404522,404942,405587,406288,407292,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,2,396483,395863,395124,394448,394961,396043,397718,400313,402353,404749,408418,412781,419227,425560,434772,443798,453651,461761,470223,477598,485358,491880,497062,502796,507853,513535,516012,516986,516275,516300,516441,517287,515847,515388,515875,519059,522838,527430,532367,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,3,181716,180629,178496,178349,178027,178483,178360,177549,176727,175763,176111,176499,176808,177536,178608,179590,180884,182116,184190,185855,187906,189501,191666,192638,193973,194711,196203,196976,197438,198107,199203,200075,200286,200614,200867,201323,201851,202343,202975,...,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,4,144594,144297,144682,145267,145949,146411,146988,147715,148149,148515,148837,149510,150185,150979,151664,152615,153583,154662,155772,156949,157887,158925,160084,161477,162718,163674,164734,165702,166669,167566,168658,169409,170150,170677,171673,172947,174399,176002,177562,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
5,5,209291,208533,207820,207529,207511,207453,206970,206278,205605,205198,204874,204793,204715,205011,205332,205884,206494,207395,208332,209225,209792,210405,210845,211414,211757,212345,212912,213284,213134,213149,213319,213603,213470,213295,213218,213394,213721,214042,214297,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0


In [29]:
# Split into training and testing data
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [30]:
# Linear Regression Model
from sklearn.linear_model import LinearRegression
model = LinearRegression()
model.fit(X_train, y_train)

LinearRegression()

In [31]:
# Score the model
model.score(X_test, y_test)

0.9999696384065571